In [1]:
import numpy as np
import polars as pl
from hazm.sentence_tokenizer import SentenceTokenizer
from hazm.word_tokenizer import WordTokenizer
import random

In [2]:
df = pl.read_excel('fa_abs.xlsx')
df.head()

fa_abs
str
"""اختلال وسواسی-…"
""" پژوهش حاضر با…"
"""پژوهش حاضر یک …"
""" پژوهش حاضر، ب…"
""" این پژوهش با …"


In [3]:
sent_token = SentenceTokenizer()

df =df.with_columns([
    pl.col('fa_abs').map_elements(
        lambda x: sent_token.tokenize(x))
])


In [4]:
text = []
for row in df.iter_rows():
    l = list(*row)
    text += l
text[:10]


['اختلال وسواسی-جبری یک اختلال ناتوان\u200cکننده است که می\u200cتواند عملکرد فرد را مختل سازد.',
 'این پژوهش با هدف بررسی اثربخشی روان\u200cنمایشگری بر کاهش علایم وسواسی-جبری و افسردگی در زنان مبتلا به اختلال وسواسی-جبری در یک طرح نیمه آزمایشی انجام گرفت.',
 'جامعه\u200cی این پژوهش شامل کلیه\u200cی زنان مبتلا به وسواسی-جبری مراجعه\u200cکننده به کلینیک\u200cهای مشاوره شهر تهران بود.',
 '24 بیمار زن مبتلا به اختلال وسواسی-جبری به صورت هدفمند و داوطلبانه انتخاب شدند و به\u200cطور تصادفی در گروه\u200cهای آزمایش و گروه کنترل قرار گرفتند.',
 'شرکت\u200cکنندگان به مقیاس وسواسی-جبری یل-براون و پرسش\u200cنامه\u200cی افسردگی بک در مراحل پیش\u200cآزمون و پس\u200cآزمون پاسخ دادند.',
 'شرکت\u200cکنندگان گروه آزمایش در پروتکل دوازده جلسه نود دقیقه\u200cای مداخله روان\u200cنمایشگری شرکت کردند و گروه کنترل مداخله\u200cای دریافت نکردند.',
 'تجزیه و تحلیل داده\u200cها با استفاده از تحلیل کوواریانس و نرم\u200cافزار آماری SPSS صورت گرفت.',
 'نتایج نشان داد که با کنترل نمرات پیش\u200cآزمون، اثر گروه بر کاه

In [5]:
#splitting the punctuations from tokens
punc = '?!./,;:"()'
tokenizer = WordTokenizer()
segments = []
for sent in text:
    for token in tokenizer.tokenize(sent):
        segments_temp = []
        for char in token:
            if char in punc:
                segments_temp.append(char)
                continue
            if not segments_temp:
                segments_temp.append(char)
                continue
            if segments_temp[-1] not in punc:
                segments_temp[-1] += char
            else:
                segments_temp.append(char)
        for seg in segments_temp:
            if not any(x in seg for x in '#$%&*123456=78;9G/ IBFc-AECD؛ ،VRPXW()'):
                segments.append(seg)

print(segments[:50])

['اختلال', 'یک', 'اختلال', 'ناتوان\u200cکننده', 'است', 'که', 'می\u200cتواند', 'عملکرد', 'فرد', 'را', 'مختل', 'سازد', '.', 'این', 'پژوهش', 'با', 'هدف', 'بررسی', 'اثربخشی', 'روان\u200cنمایشگری', 'بر', 'کاهش', 'علایم', 'و', 'افسردگی', 'در', 'زنان', 'مبتلا', 'به', 'اختلال', 'در', 'یک', 'طرح', 'نیمه', 'آزمایشی', 'انجام', 'گرفت', '.', 'جامعه\u200cی', 'این', 'پژوهش', 'شامل', 'کلیه\u200cی', 'زنان', 'مبتلا', 'به', 'مراجعه\u200cکننده', 'به', 'کلینیک\u200cهای', 'مشاوره']


In [6]:
vocab = list(set(segments))+['<s>', '</s>']
vocab.sort()
print(vocab[:50])

['!', '"', "'", '+', ',', '.', '0', '00', '000', ':', '<', '</s>', '<br', '<s>', '<span', '>', '><br', '>\xa0', '>\xa0دین', '>آسیب', '>این', '>بیماران', '>تیپ', '>در', '>دریک', '>روش', '>زمینه', '>زمینه\xa0', '>مقدمه', '>مقدمه\xa0', '>مواد', '>موقعیت', '>نتایج', '>نتیجه', '>هدف', '>همچنین', '>پژوهش', '>یافته', '?', 'H', 'HLM', 'HML', 'HMT', 'HS', 'Happiness', 'Hardy', 'Harris', 'Havlena', 'Hazan', 'Health']


In [7]:
#inserting sentence start and end paddings naively just looking at dots
def padding(segments: list[str] , n: int) -> list[str]:
    start_of_sentence = True
    with_padding = []
    for token in segments:
        if start_of_sentence:
            for i in range(n-1):
                with_padding.append('<s>')
            start_of_sentence = False
        with_padding.append(token)
        if token == "." :
            for i in range(n-1):
                with_padding.append('</s>')
            start_of_sentence = True
    return with_padding

print(padding(segments , 4)[:150])

['<s>', '<s>', '<s>', 'اختلال', 'یک', 'اختلال', 'ناتوان\u200cکننده', 'است', 'که', 'می\u200cتواند', 'عملکرد', 'فرد', 'را', 'مختل', 'سازد', '.', '</s>', '</s>', '</s>', '<s>', '<s>', '<s>', 'این', 'پژوهش', 'با', 'هدف', 'بررسی', 'اثربخشی', 'روان\u200cنمایشگری', 'بر', 'کاهش', 'علایم', 'و', 'افسردگی', 'در', 'زنان', 'مبتلا', 'به', 'اختلال', 'در', 'یک', 'طرح', 'نیمه', 'آزمایشی', 'انجام', 'گرفت', '.', '</s>', '</s>', '</s>', '<s>', '<s>', '<s>', 'جامعه\u200cی', 'این', 'پژوهش', 'شامل', 'کلیه\u200cی', 'زنان', 'مبتلا', 'به', 'مراجعه\u200cکننده', 'به', 'کلینیک\u200cهای', 'مشاوره', 'شهر', 'تهران', 'بود', '.', '</s>', '</s>', '</s>', '<s>', '<s>', '<s>', 'بیمار', 'زن', 'مبتلا', 'به', 'اختلال', 'به', 'صورت', 'هدفمند', 'و', 'داوطلبانه', 'انتخاب', 'شدند', 'و', 'به\u200cطور', 'تصادفی', 'در', 'گروه\u200cهای', 'آزمایش', 'و', 'گروه', 'کنترل', 'قرار', 'گرفتند', '.', '</s>', '</s>', '</s>', '<s>', '<s>', '<s>', 'شرکت\u200cکنندگان', 'به', 'مقیاس', 'و', 'پرسش\u200cنامه\u200cی', 'افسردگی', 'بک', 'در', 'مراحل', 

In [8]:
#making a function that produces ngrams given n
def ngram(segments: list[str] , n: int) -> list[tuple]:
    seg = padding(segments , n)
    ngrams = []
    i = 0
    while i < (len(seg)-n):
        ngrams.append(tuple(seg[i:i+n]))
        i += 1
    return ngrams
print(ngram(segments , 3)[:30])        

[('<s>', '<s>', 'اختلال'), ('<s>', 'اختلال', 'یک'), ('اختلال', 'یک', 'اختلال'), ('یک', 'اختلال', 'ناتوان\u200cکننده'), ('اختلال', 'ناتوان\u200cکننده', 'است'), ('ناتوان\u200cکننده', 'است', 'که'), ('است', 'که', 'می\u200cتواند'), ('که', 'می\u200cتواند', 'عملکرد'), ('می\u200cتواند', 'عملکرد', 'فرد'), ('عملکرد', 'فرد', 'را'), ('فرد', 'را', 'مختل'), ('را', 'مختل', 'سازد'), ('مختل', 'سازد', '.'), ('سازد', '.', '</s>'), ('.', '</s>', '</s>'), ('</s>', '</s>', '<s>'), ('</s>', '<s>', '<s>'), ('<s>', '<s>', 'این'), ('<s>', 'این', 'پژوهش'), ('این', 'پژوهش', 'با'), ('پژوهش', 'با', 'هدف'), ('با', 'هدف', 'بررسی'), ('هدف', 'بررسی', 'اثربخشی'), ('بررسی', 'اثربخشی', 'روان\u200cنمایشگری'), ('اثربخشی', 'روان\u200cنمایشگری', 'بر'), ('روان\u200cنمایشگری', 'بر', 'کاهش'), ('بر', 'کاهش', 'علایم'), ('کاهش', 'علایم', 'و'), ('علایم', 'و', 'افسردگی'), ('و', 'افسردگی', 'در')]


In [9]:
def gram_counts(segments: list[str] , n: int):
    count_dict = {}
    
    for gram in ngram(segments , n):
        if gram in count_dict:
            count_dict[gram] += 1
        else:
            count_dict[gram] = 1
    if n > 1:        
        for gram in ngram(segments , n-1):
            if gram in count_dict:
                count_dict[gram] += 1
            else:
                count_dict[gram] = 1
    return count_dict
    
counts = gram_counts(segments , 3)
n = 0
for i in counts.__iter__():
    n += 1 
    if n > 10:
        break
    print(i , counts[i])

('<s>', '<s>', 'اختلال') 103
('<s>', 'اختلال', 'یک') 3
('اختلال', 'یک', 'اختلال') 2
('یک', 'اختلال', 'ناتوان\u200cکننده') 1
('اختلال', 'ناتوان\u200cکننده', 'است') 1
('ناتوان\u200cکننده', 'است', 'که') 1
('است', 'که', 'می\u200cتواند') 1
('که', 'می\u200cتواند', 'عملکرد') 1
('می\u200cتواند', 'عملکرد', 'فرد') 1
('عملکرد', 'فرد', 'را') 1


In [10]:
class Prob_gen():
    
    def __init__(self, segments: str , n: int):
        self.counts = gram_counts(segments , n)
        self.n = n

    def gramProb(self , gram: tuple):
        try:
            return self.counts[gram]/self.counts[tuple(gram[0:-1])]
        except:
            return 0.001
    
    def sentProbs(self , sentence: list[str]):
        prob = []
        for i in range(len(sentence)-self.n+1):
            prob.append(self.gramProb(tuple(sentence[i:i+self.n])))
        return prob   
    

In [11]:
print('total segments: ' ,len(segments))
train = len(segments)*80//100
print('train 80% :' , train)
test = len(segments)*90//100
print('dev 10% :' , train, ':' , test)
print('test 10% :' , len(segments)*90//100, ':')

total segments:  1390393
train 80% : 1112314
dev 10% : 1112314 : 1251353
test 10% : 1251353 :


In [12]:
probgen = Prob_gen(segments[:train] , 3)

In [13]:
def perplexity(probgen: Prob_gen , test_set: list[str]):
    return np.prod([(1/p)**(1/len(test_set)) for p in probgen.sentProbs(test_set)])

In [14]:
print("grams     peplexity")
for i in range(1,7):
    probgen = Prob_gen(segments[:train] , i)
    print(i , '       ' , round(perplexity(probgen , segments[train:test])))

grams     peplexity
1         1000
2         57
3         68
4         159
5         305
6         464


In [15]:
probgen = Prob_gen(segments[:train] , 2)
perplexity(probgen , segments[test:])

55.751514547458335

In [16]:
def sent_gen(sentence_count: int):
    
    init = ['<s>']
    out = []
    
    while sentence_count > 0:
        
        temp_dict = {}
        
        for gram in probgen.counts:
            if (init[-1] == gram[0]) & (len(gram) == 2):
                temp_dict.update({gram: probgen.counts[gram]})
        s = sum(temp_dict.values())
        pop = [x for x in temp_dict.keys()]
        prob = [probgen.gramProb(x) for x in temp_dict.keys()]
        init.append(random.choices(pop , prob)[0][1])
        if init[-1] == '.':
            sentence_count -= 1
            
    for t in init:
        if t not in ['<s>','</s>']:
            out.append(t)
    return out                

In [17]:
print(*sent_gen(5))

مراجعان به احساسات و بررسی قرار گرفتند و اندیشه سیمونز و اجتماعی ماسیا و پیگیری با تاثیر مشترک انسانی فنی و صمیمیت را با پیامبر صلی الله است که مادران شده با استفاده از این پژوهش از نوع صوری موردبررسی با صمیمیت و شاخص آلفای است .  جامعه مورد کنکاش قرار گرفتند . دل بستگی بر نگرش های اولیه در ارتباط مفهوم خویشتن بر حساسیت اضطرابی بود . ال اس شرکت به عنوان ابزار اندازه گیری هدفمند آزمودنی ها نشان می باشدو به یافته های جامعه پژوهش پرسشنامه های زوجین گروه کنترل بود . میل به والدین و به لوکمی حاد لنفوبلاستی مفید و غیرهم جنس را نشان داد که در سنین تا در رابطه دارد .


In [18]:
def next_word(sentence: str):

    last = sentence.split()[-1]

    next_word = ''
    prob = 0
    for gram in probgen.counts:
            if (last == gram[0]) & (len(gram) == 2):
                if probgen.counts[gram] > prob:
                     next_word = gram[1]
                     prob = probgen.counts[gram]
    
    print(sentence, next_word)

In [19]:
next_word('درد')

درد مزمن
